In [32]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [21]:
year = '2019'
df = pd.read_csv('scraping/flickr_data_'+year+'.csv')
df_no_dup = df.drop_duplicates(ignore_index=True)
df_no_dup.to_csv('scraping/flickr_data_'+year+'_no_dup.csv', index=False)
df_no_dup

,ID,Owner,Secret,Server,Title,Description,Year,Month,Day,Time,Hashtags,Latitude,Longitude
0,49306796181,100571744@N07,df249100ab,65535,Grüne Mamba,"Please don't use this image on websites, blogs...",2019,12,31,21:50:26,troparium hamburg grünemamba mamba giftig gefa...,0.000000,0.000000
1,49306999322,96034594@N03,b19cded762,65535,Feuerwerk 5,New Years Fireworks 2019/2020,2019,12,31,21:53:35,olympuspenf hamburg farmsenberne silvesterfeue...,0.000000,0.000000
2,49306999207,96034594@N03,aebb62b8f9,65535,Feuerwerk 3,New Years Fireworks 2019/2020,2019,12,31,21:53:36,olympuspenf hamburg farmsenberne silvesterfeue...,0.000000,0.000000
3,49306297293,96034594@N03,abcc75bf5e,65535,Feuerwerk 7,New Years Fireworks 2019/2020,2019,12,31,21:53:36,olympuspenf hamburg farmsenberne silvesterfeue...,0.000000,0.000000
4,49305904733,81182384@N02,523ec04c8c,65535,D-BTEX 51 80 98-80 138-5 DDm 915.0 Hamburg-Alt...,BTE 1796 Lörrach Gbf ARZ – Hamburg-Altona,2019,12,31,20:27:23,hamburg deutschland,53.554465,9.934859
...,...,...,...,...,...,...,...,...,...,...,...,...,...
61595,45825802244,37215607@N04,13c44873b1,65535,-,NaN,2019,1,1,04:11:26,hamburg altona people bw sw keyboard,0.000000,0.000000
61596,45825794834,37215607@N04,c69805d10a,65535,-,NaN,2019,1,1,04:11:27,hamburg altona people bw sw cafe cafelagos,0.000000,0.000000
61597,45825793964,37215607@N04,4afdf2f5e9,65535,-,NaN,2019,1,1,04:11:25,hamburg altona people bw sw exhibition ausstel...,0.000000,0.000000
61598,45825793224,37215607@N04,d95a1fc91e,65535,-,NaN,2019,1,1,04:11:26,hamburg altona people bw sw,0.000000,0.000000


In [61]:
def summarize_gallery(data):
    data = data.replace(np.nan, '', regex=True)
    # Retrieve the additional images for existing posts
    mask = data.duplicated(subset=['Hashtags', 'Description', 'Year', 'Month', 'Day', 'Longitude', 'Latitude'], keep='first')
    dup = data.loc[mask]
    # Keep the first post and remove the duplicates
    data_no_dup = data.drop_duplicates(subset=['Hashtags', 'Description', 'Year', 'Month', 'Day', 'Longitude', 'Latitude'], keep='first').copy()
    data_no_dup['ID'] = data_no_dup['ID'].apply(lambda x:[x])
    errors = pd.DataFrame()
    # append additional images to ID list per post
    for _, row in tqdm(dup.iterrows(), total=dup.shape[0]):
        orig = data_no_dup.loc[(row.Longitude == data_no_dup.Longitude) & 
                              (row.Latitude == data_no_dup.Latitude) &
                              (row.Hashtags == data_no_dup.Hashtags) &
                              (row.Description == data_no_dup.Description) &
                              (row.Year == data_no_dup.Year) &
                              (row.Month == data_no_dup.Month) &
                              (row.Day == data_no_dup.Day)]
        if orig.empty:
            print(row)
            print(data_no_dup.loc[(row.Description == data_no_dup.Description)])
            errors.append(row)
            continue
        data_no_dup.loc[:,'ID'][orig.index[0]] += [row.ID]
    return data_no_dup

In [66]:
for y in ['2018', '2019']:
    df = pd.read_csv('scraping/flickr_data_'+y+'_no_dup.csv')
    df_no_dup = summarize_gallery(df)
    df_no_dup.to_csv('scraping/flickr_data_'+y+'_gallery.csv', index=False)

  0%|                                                                                                                                                                                    | 0/45224 [00:00<?, ?it/s]/tmp/ipykernel_283124/2397274820.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_no_dup.loc[:,'ID'][orig.index[0]] += [row.ID]
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 33399/33399 [04:26<00:00, 125.21it/s]


In [65]:
df_no_dup.to_csv('scraping/flickr_data_2017_gallery.csv', index=False)